# RankNet

In [1]:
import torch

In [2]:
%%latex
$$C_{ij}=C(o_{ij})=-\bar{P_{ij}}log(P_{ij})-(1-\bar{P_{ij}})log(1-P_{ij})$$

<IPython.core.display.Latex object>

In [3]:
%%latex
$$o_{ij}=f(x_i)-f(x_j)$$

<IPython.core.display.Latex object>

In [4]:
%%latex
$$P_{ij}=\frac{e^{o_{ij}}}{1+e^{o_{ij}}}$$

<IPython.core.display.Latex object>

In [5]:
torch.sigmoid()

TypeError: sigmoid() missing 1 required positional arguments: "input"

In [6]:
%%latex
$$\text{out}_{i} = \frac{1}{1 + e^{-\text{input}_{i}}}$$

<IPython.core.display.Latex object>

In [7]:
class RankNet(torch.nn.Module):
    def __init__(self, num_input_features, hidden_dim=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            torch.nn.Linear(num_input_features, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1),
        )
        
        self.out_activation = torch.nn.Sigmoid()

    def forward(self, input_1, input_2):
        logits_1 = self.predict(input_1)
        logits_2 = self.predict(input_2)
        
        logits_diff = logits_1 - logits_2
        out = self.out_activation(logits_diff)

        return out
    
    def predict(self, inp):
        logits = self.model(inp)
        return logits

In [18]:
ranknet_model = RankNet(num_input_features=10)

In [19]:
inp_1, inp_2 = torch.rand(4, 10), torch.rand(4, 10)
# batch_size x input_dim
inp_2

tensor([[0.4737, 0.3166, 0.5947, 0.9229, 0.8446, 0.3150, 0.4467, 0.7716, 0.4611,
         0.7093],
        [0.9518, 0.4163, 0.1700, 0.1561, 0.0695, 0.6232, 0.6392, 0.9687, 0.2451,
         0.9066],
        [0.9838, 0.0995, 0.8838, 0.6342, 0.2499, 0.5543, 0.5031, 0.6127, 0.3663,
         0.9750],
        [0.8561, 0.2213, 0.6429, 0.4844, 0.3372, 0.2926, 0.8280, 0.2087, 0.4599,
         0.0081]])

In [20]:
preds = ranknet_model(inp_1, inp_2)
preds

tensor([[0.5138],
        [0.4839],
        [0.4896],
        [0.5484]], grad_fn=<SigmoidBackward0>)

In [21]:
first_linear_layer = ranknet_model.model[0]

In [22]:
first_linear_layer.weight.grad

In [23]:
criterion = torch.nn.BCELoss()
loss = criterion(preds, torch.ones_like(preds))
loss.backward()

In [24]:
first_linear_layer.weight.grad

tensor([[ 2.5911e-02, -3.4822e-02,  3.9106e-03,  2.3812e-02,  5.7420e-03,
         -4.7057e-04, -1.6776e-03,  1.2583e-02, -2.2965e-03,  3.5100e-02],
        [ 1.7269e-02,  8.9985e-03,  3.9080e-03,  1.6665e-02,  9.8740e-03,
          4.6578e-03,  1.9981e-02,  1.5860e-04,  9.3742e-03,  9.4872e-04],
        [ 1.6274e-02, -1.1552e-02,  2.8733e-03,  1.5211e-02,  5.5440e-03,
          1.2973e-03,  5.7065e-03,  5.2672e-03,  2.0515e-03,  1.4855e-02],
        [-1.9514e-03, -1.1457e-02, -1.9429e-02, -2.1092e-03, -1.1832e-02,
         -9.5252e-03, -6.3733e-03, -1.9986e-02, -1.1543e-02, -3.3493e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 5.3690e-03, -7.2154e-03,  8.1030e-04,  4.9340e-03,  1.1898e-03,
         -9.7507e-05, -3.4760e-04,  2.6074e-03, -4.7585e-04,  7.2729e-03],
        [ 3.9103e-02, -5.2550e-02,  5.9015e-03,  3.5935e-02,  8.6653e-03,
         -7.1016e-04, -2.5316e-0

In [25]:
ranknet_model.zero_grad()

# ListNet

In [1]:
from itertools import combinations
import numpy as np

from utils.metrics import ndcg, num_swapped_pairs

In [40]:
class ListNet(torch.nn.Module):
    def __init__(self, num_input_features, hidden_dim=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            torch.nn.Linear(num_input_features, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1),
        )


    def forward(self, input_1):
        logits = self.model(input_1)
        return logits


In [41]:
%%latex
$$CE = -\sum ^{N}_{j=1} (P_y^i(j) * log(P_z^i(j)))$$

<IPython.core.display.Latex object>

In [42]:
%%latex
$$\text{Softmax}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$$

<IPython.core.display.Latex object>

In [43]:
def listnet_ce_loss(y_i, z_i):
    """
    y_i: (n_i, 1) GT
    z_i: (n_i, 1) preds
    """

    P_y_i = torch.softmax(y_i, dim=0)
    P_z_i = torch.softmax(z_i, dim=0)
    return -torch.sum(P_y_i * torch.log(P_z_i))

def listnet_kl_loss(y_i, z_i):
    """
    y_i: (n_i, 1) GT
    z_i: (n_i, 1) preds
    """
    P_y_i = torch.softmax(y_i, dim=0)
    P_z_i = torch.softmax(z_i, dim=0)
    return -torch.sum(P_y_i * torch.log(P_z_i/P_y_i))


def make_dataset(N_train, N_valid, vector_dim):
    fake_weights = torch.randn(vector_dim, 1)

    X_train = torch.randn(N_train, vector_dim)
    X_valid = torch.randn(N_valid, vector_dim)

    ys_train_score = torch.mm(X_train, fake_weights)
    ys_train_score += torch.randn_like(ys_train_score)

    ys_valid_score = torch.mm(X_valid, fake_weights)
    ys_valid_score += torch.randn_like(ys_valid_score)

#     bins = [-1, 1]  # 3 relevances
    bins = [-1, 0, 1, 2]  # 5 relevances
    ys_train_rel = torch.Tensor(
        np.digitize(ys_train_score.clone().detach().numpy(), bins=bins)
    )
    ys_valid_rel = torch.Tensor(
        np.digitize(ys_valid_score.clone().detach().numpy(), bins=bins)
    )

    return X_train, X_valid, ys_train_rel, ys_valid_rel

In [44]:
N_train = 1000
N_valid = 500

vector_dim = 100
epochs = 2

batch_size = 16

X_train, X_valid, ys_train, ys_valid = make_dataset(N_train, N_valid, vector_dim)

net = ListNet(num_input_features=vector_dim)
opt = torch.optim.Adam(net.parameters())

In [45]:
torch.unique(ys_train)

tensor([0., 1., 2., 3., 4.])

In [46]:
for epoch in range(epochs):
    idx = torch.randperm(N_train)

    X_train = X_train[idx]
    ys_train = ys_train[idx]

    cur_batch = 0
    for it in range(N_train // batch_size):
        batch_X = X_train[cur_batch: cur_batch + batch_size]
        batch_ys = ys_train[cur_batch: cur_batch + batch_size]
        cur_batch += batch_size

        opt.zero_grad()
        if len(batch_X) > 0:
            batch_pred = net(batch_X)
            batch_loss = listnet_kl_loss(batch_ys, batch_pred)
#             batch_loss = listnet_ce_loss(batch_ys, batch_pred)
            batch_loss.backward(retain_graph=True)
            opt.step()

        if it % 10 == 0:
            with torch.no_grad():
                valid_pred = net(X_valid)
                valid_swapped_pairs = num_swapped_pairs(ys_valid, valid_pred)
                ndcg_score = ndcg(ys_valid, valid_pred)
            print(f"epoch: {epoch + 1}.\tNumber of swapped pairs: " 
                  f"{valid_swapped_pairs}/{N_valid * (N_valid - 1) // 2}\t"
                  f"nDCG: {ndcg_score.item():.4f}")

epoch: 1.	Number of swapped pairs: 0/124750	nDCG: 1.0000
epoch: 1.	Number of swapped pairs: 0/124750	nDCG: 1.0000
epoch: 1.	Number of swapped pairs: 0/124750	nDCG: 1.0000
epoch: 1.	Number of swapped pairs: 0/124750	nDCG: 1.0000
epoch: 1.	Number of swapped pairs: 0/124750	nDCG: 1.0000
epoch: 1.	Number of swapped pairs: 0/124750	nDCG: 1.0000
epoch: 1.	Number of swapped pairs: 0/124750	nDCG: 1.0000
epoch: 2.	Number of swapped pairs: 0/124750	nDCG: 1.0000
epoch: 2.	Number of swapped pairs: 0/124750	nDCG: 1.0000
epoch: 2.	Number of swapped pairs: 0/124750	nDCG: 1.0000
epoch: 2.	Number of swapped pairs: 0/124750	nDCG: 1.0000
epoch: 2.	Number of swapped pairs: 0/124750	nDCG: 1.0000
epoch: 2.	Number of swapped pairs: 0/124750	nDCG: 1.0000
epoch: 2.	Number of swapped pairs: 0/124750	nDCG: 1.0000
